# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
%matplotlib inline
!jupyter nbextension enable --py gmaps
!jupyter nbextension enable --py widgetsnbextension

# Import API key
from api_keys import g_key



Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

output_data_file = "weather_output/weather_data.csv"

weather_data = pd.read_csv(output_data_file)

weather_data_df = weather_data.drop(columns=["Unnamed: 0"])
weather_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Ponta do Sol,20,PT,1580537643,51,32.67,-17.10,66.00,2.24
1,Paithan,20,IN,1580537938,49,19.48,75.38,73.40,5.82
2,Katsuura,20,JP,1580537635,34,35.13,140.30,55.99,6.93
3,Qaanaaq,100,GL,1580537608,99,77.48,-69.36,-23.48,0.89
4,Esperance,75,AU,1580537199,38,-33.87,121.90,73.40,14.99
...,...,...,...,...,...,...,...,...,...
580,Puerto Berrio,67,CO,1580538620,96,6.49,-74.40,72.36,2.80
581,Prince Rupert,90,CA,1580538622,86,54.32,-130.32,37.40,11.41
582,Manavalakurichi,20,IN,1580538623,66,8.13,77.30,86.00,6.93
583,Turukhansk,100,RU,1580538624,96,65.82,87.98,-20.72,7.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:

locations = weather_data_df[["Lat", "Lng"]]
humidity = weather_data_df["Humidity"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)
                                 
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_df = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])
max_temp = (new_df["Max Temp"] <= 80) & (new_df["Max Temp"] > 70)
wind_speed = new_df["Wind Speed"] < 10
cloudiness = new_df["Cloudiness"] == 0

new_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
199,Torres,75.47,6.22,0
201,La Plata,73.40,4.70,0
213,Catuday,77.32,2.17,0
264,Bodden Town,78.01,1.99,0
279,Haldia,76.39,2.66,0
333,Neuquen,73.40,9.17,0
437,Cordoba,72.00,2.24,0
438,Sojitra,75.20,6.93,0
521,Mogok,74.23,3.85,0
554,Gweta,75.45,8.34,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(weather_data_df, columns=["City", "Country", "Lat", "Lng", "Max Temp", "Wind Speed", "Cloudiness"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Hotel Name
0,Ponta do Sol,PT,32.67,-17.10,66.00,2.24,20,
1,Paithan,IN,19.48,75.38,73.40,5.82,20,
2,Katsuura,JP,35.13,140.30,55.99,6.93,20,
3,Qaanaaq,GL,77.48,-69.36,-23.48,0.89,100,
4,Esperance,AU,-33.87,121.90,73.40,14.99,75,
...,...,...,...,...,...,...,...,...
580,Puerto Berrio,CO,6.49,-74.40,72.36,2.80,67,
581,Prince Rupert,CA,54.32,-130.32,37.40,11.41,90,
582,Manavalakurichi,IN,8.13,77.30,86.00,6.93,20,
583,Turukhansk,RU,65.82,87.98,-20.72,7.36,100,


In [10]:
params = { "types": "lodging",
          "radius":5000,"key": g_key}

# Use the lat/lng 
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
#parameters 
params["location"] = f"{lat},{lng}"

# Base_url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
name_address = requests.get(base_url, params=params).json()




In [8]:
hotel_df

,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Hotel Name
0,Ponta do Sol,PT,32.67,-17.10,66.00,2.24,20,
1,Paithan,IN,19.48,75.38,73.40,5.82,20,
2,Katsuura,JP,35.13,140.30,55.99,6.93,20,
3,Qaanaaq,GL,77.48,-69.36,-23.48,0.89,100,
4,Esperance,AU,-33.87,121.90,73.40,14.99,75,
...,...,...,...,...,...,...,...,...
580,Puerto Berrio,CO,6.49,-74.40,72.36,2.80,67,
581,Prince Rupert,CA,54.32,-130.32,37.40,11.41,90,
582,Manavalakurichi,IN,8.13,77.30,86.00,6.93,20,
583,Turukhansk,RU,65.82,87.98,-20.72,7.36,100,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))